#**IMPORTING THE LIBRIRIES**

In [219]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.dummy import DummyRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import make_column_transformer
from statsmodels.stats.outliers_influence import variance_inflation_factor

#**1. ETL**

##**1.1. Extract**

### 1.1.1. Reading the Data

In [220]:
url = '/content/drive/MyDrive/Challenge telecom x/TelecomX_Data_clean.csv'
datos = pd.read_csv(url)

In [221]:
datos.head(3)

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,...,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total
0,0002-ORFBO,No,Female,0,Yes,Yes,9,Yes,No,DSL,...,Yes,No,Yes,Yes,No,One year,Yes,Mailed check,65.6,593.30
1,0003-MKNFE,No,Male,0,No,No,9,Yes,Yes,DSL,...,No,No,No,No,Yes,Month-to-month,No,Mailed check,59.9,542.40
2,0004-TLHLJ,Yes,Male,0,No,No,4,Yes,No,Fiber optic,...,No,Yes,No,No,No,Month-to-month,Yes,Electronic check,73.9,280.85


### 1.1.2. Transform

In [222]:
datos.describe()

,SeniorCitizen,tenure,Charges.Monthly,Charges.Total
count,7043.000000,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692,2279.734304
std,0.368612,24.559481,30.090047,2266.794470
min,0.000000,0.000000,18.250000,0.000000
25%,0.000000,9.000000,35.500000,398.550000
50%,0.000000,29.000000,70.350000,1394.550000
75%,0.000000,55.000000,89.850000,3786.600000
max,1.000000,72.000000,118.750000,8684.800000


In [223]:
datos.describe(include='O')

,customerID,Churn,gender,Partner,Dependents,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod
count,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043,7043
unique,7043,2,2,2,2,2,3,3,3,3,3,3,3,3,3,2,4
top,9995-HOTOH,No,Male,No,No,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check
freq,1,5174,3555,3641,4933,6361,3390,3096,3498,3088,3095,3473,2810,2785,3875,4171,2365


In [224]:
datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   Churn             7043 non-null   object 
 2   gender            7043 non-null   object 
 3   SeniorCitizen     7043 non-null   int64  
 4   Partner           7043 non-null   object 
 5   Dependents        7043 non-null   object 
 6   tenure            7043 non-null   int64  
 7   PhoneService      7043 non-null   object 
 8   MultipleLines     7043 non-null   object 
 9   InternetService   7043 non-null   object 
 10  OnlineSecurity    7043 non-null   object 
 11  OnlineBackup      7043 non-null   object 
 12  DeviceProtection  7043 non-null   object 
 13  TechSupport       7043 non-null   object 
 14  StreamingTV       7043 non-null   object 
 15  StreamingMovies   7043 non-null   object 
 16  Contract          7043 non-null   object 


In [225]:
for col in datos:
  print(col, datos[col].unique())

customerID ['0002-ORFBO' '0003-MKNFE' '0004-TLHLJ' ... '9992-UJOEL' '9993-LHIEB'
 '9995-HOTOH']
Churn ['No' 'Yes']
gender ['Female' 'Male']
SeniorCitizen [0 1]
Partner ['Yes' 'No']
Dependents ['Yes' 'No']
tenure [ 9  4 13  3 71 63  7 65 54 72  5 56 34  1 45 50 23 55 26 69 37 49 66 67
 20 43 59 12 27  2 25 29 14 35 64 39 40 11  6 30 70 57 58 16 32 33 10 21
 61 15 44 22 24 19 47 62 46 52  8 60 48 28 41 53 68 31 36 17 18 51 38 42
  0]
PhoneService ['Yes' 'No']
MultipleLines ['No' 'Yes' 'No phone service']
InternetService ['DSL' 'Fiber optic' 'No']
OnlineSecurity ['No' 'Yes' 'No internet service']
OnlineBackup ['Yes' 'No' 'No internet service']
DeviceProtection ['No' 'Yes' 'No internet service']
TechSupport ['Yes' 'No' 'No internet service']
StreamingTV ['Yes' 'No' 'No internet service']
StreamingMovies ['No' 'Yes' 'No internet service']
Contract ['One year' 'Month-to-month' 'Two year']
PaperlessBilling ['Yes' 'No']
PaymentMethod ['Mailed check' 'Electronic check' 'Credit card (automatic)'

#### 1.1.2.1. Transformando columnas categoricas

In [226]:
columnas = datos.columns
columnas

Index(['customerID', 'Churn', 'gender', 'SeniorCitizen', 'Partner',
       'Dependents', 'tenure', 'PhoneService', 'MultipleLines',
       'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
       'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract',
       'PaperlessBilling', 'PaymentMethod', 'Charges.Monthly',
       'Charges.Total'],
      dtype='object')

In [227]:
categoricas = ['Churn','gender','Partner', 'Dependents', 'PhoneService',
               'PaperlessBilling','MultipleLines','InternetService','OnlineSecurity','OnlineBackup',
               'DeviceProtection', 'TechSupport', 'StreamingTV',
               'StreamingMovies','Contract','PaymentMethod']

In [228]:
one_hot = make_column_transformer(
    (OneHotEncoder(drop='first'), categoricas),
    remainder='passthrough',
    sparse_threshold=0,
    force_int_remainder_cols=False
)

In [229]:
datos_encode = one_hot.fit_transform(datos)

In [230]:
one_hot.get_feature_names_out(columnas)

array(['onehotencoder__Churn_Yes', 'onehotencoder__gender_Male',
       'onehotencoder__Partner_Yes', 'onehotencoder__Dependents_Yes',
       'onehotencoder__PhoneService_Yes',
       'onehotencoder__PaperlessBilling_Yes',
       'onehotencoder__MultipleLines_No phone service',
       'onehotencoder__MultipleLines_Yes',
       'onehotencoder__InternetService_Fiber optic',
       'onehotencoder__InternetService_No',
       'onehotencoder__OnlineSecurity_No internet service',
       'onehotencoder__OnlineSecurity_Yes',
       'onehotencoder__OnlineBackup_No internet service',
       'onehotencoder__OnlineBackup_Yes',
       'onehotencoder__DeviceProtection_No internet service',
       'onehotencoder__DeviceProtection_Yes',
       'onehotencoder__TechSupport_No internet service',
       'onehotencoder__TechSupport_Yes',
       'onehotencoder__StreamingTV_No internet service',
       'onehotencoder__StreamingTV_Yes',
       'onehotencoder__StreamingMovies_No internet service',
       'oneh

In [231]:
datos_encode_df = pd.DataFrame(datos_encode, columns=one_hot.get_feature_names_out(columnas))

In [232]:
datos_encode_df.head()

,onehotencoder__Churn_Yes,onehotencoder__gender_Male,onehotencoder__Partner_Yes,onehotencoder__Dependents_Yes,onehotencoder__PhoneService_Yes,onehotencoder__PaperlessBilling_Yes,onehotencoder__MultipleLines_No phone service,onehotencoder__MultipleLines_Yes,onehotencoder__InternetService_Fiber optic,onehotencoder__InternetService_No,...,onehotencoder__Contract_One year,onehotencoder__Contract_Two year,onehotencoder__PaymentMethod_Credit card (automatic),onehotencoder__PaymentMethod_Electronic check,onehotencoder__PaymentMethod_Mailed check,remainder__customerID,remainder__SeniorCitizen,remainder__tenure,remainder__Charges.Monthly,remainder__Charges.Total
0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0002-ORFBO,0,9,65.6,593.3
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0003-MKNFE,0,9,59.9,542.4
2,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0004-TLHLJ,0,4,73.9,280.85
3,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0011-IGKFF,1,13,98.0,1237.85
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0013-EXCHZ,1,3,83.9,267.4


In [233]:
datos_encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 32 columns):
 #   Column                                                Non-Null Count  Dtype 
---  ------                                                --------------  ----- 
 0   onehotencoder__Churn_Yes                              7043 non-null   object
 1   onehotencoder__gender_Male                            7043 non-null   object
 2   onehotencoder__Partner_Yes                            7043 non-null   object
 3   onehotencoder__Dependents_Yes                         7043 non-null   object
 4   onehotencoder__PhoneService_Yes                       7043 non-null   object
 5   onehotencoder__PaperlessBilling_Yes                   7043 non-null   object
 6   onehotencoder__MultipleLines_No phone service         7043 non-null   object
 7   onehotencoder__MultipleLines_Yes                      7043 non-null   object
 8   onehotencoder__InternetService_Fiber optic            7043 non-null 

#### 1.1.2.2. Eliminando columnas irrelevantes

**Eliminando customerID, ya que no es una variable que nos aporte informacion relevante**

In [234]:
datos_encode_df = datos_encode_df.drop(columns = 'remainder__customerID')
datos_encode_df.head()

,onehotencoder__Churn_Yes,onehotencoder__gender_Male,onehotencoder__Partner_Yes,onehotencoder__Dependents_Yes,onehotencoder__PhoneService_Yes,onehotencoder__PaperlessBilling_Yes,onehotencoder__MultipleLines_No phone service,onehotencoder__MultipleLines_Yes,onehotencoder__InternetService_Fiber optic,onehotencoder__InternetService_No,...,onehotencoder__StreamingMovies_Yes,onehotencoder__Contract_One year,onehotencoder__Contract_Two year,onehotencoder__PaymentMethod_Credit card (automatic),onehotencoder__PaymentMethod_Electronic check,onehotencoder__PaymentMethod_Mailed check,remainder__SeniorCitizen,remainder__tenure,remainder__Charges.Monthly,remainder__Charges.Total
0,0.0,0.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0,9,65.6,593.3
1,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,1.0,0,9,59.9,542.4
2,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0,4,73.9,280.85
3,1.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,1.0,0.0,1,13,98.0,1237.85
4,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1,3,83.9,267.4


**necesitamos convertir nuestros datos que ahora estan en tipo object a tipo numerio int o float**

In [235]:
datos_encode_df = datos_encode_df.apply(pd.to_numeric, errors='ignore')

/tmp/ipython-input-45323555.py:1: FutureWarning: errors='ignore' is deprecated and will raise in a future version. Use to_numeric without passing `errors` and catch exceptions explicitly instead
  datos_encode_df = datos_encode_df.apply(pd.to_numeric, errors='ignore')


In [236]:
datos_encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 31 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   onehotencoder__Churn_Yes                              7043 non-null   float64
 1   onehotencoder__gender_Male                            7043 non-null   float64
 2   onehotencoder__Partner_Yes                            7043 non-null   float64
 3   onehotencoder__Dependents_Yes                         7043 non-null   float64
 4   onehotencoder__PhoneService_Yes                       7043 non-null   float64
 5   onehotencoder__PaperlessBilling_Yes                   7043 non-null   float64
 6   onehotencoder__MultipleLines_No phone service         7043 non-null   float64
 7   onehotencoder__MultipleLines_Yes                      7043 non-null   float64
 8   onehotencoder__InternetService_Fiber optic            7043

**Revisando la correlacion con nuestra variable objetivo Churn**

In [237]:
datos_encode_df.corr()['onehotencoder__Churn_Yes'].sort_values()

,onehotencoder__Churn_Yes
remainder__tenure,-0.352229
onehotencoder__Contract_Two year,-0.302253
onehotencoder__OnlineBackup_No internet service,-0.227890
onehotencoder__DeviceProtection_No internet service,-0.227890
onehotencoder__TechSupport_No internet service,-0.227890
onehotencoder__StreamingMovies_No internet service,-0.227890
onehotencoder__OnlineSecurity_No internet service,-0.227890
onehotencoder__InternetService_No,-0.227890
onehotencoder__StreamingTV_No internet service,-0.227890
remainder__Charges.Total,-0.198324


**De acuerdo con la tabla anterior de correlaciones, podemos concluir que hay algunas columnas que no aportan peso a nuestro analisis, por lo general las correlaciones muy cerca a 0 y las mayores de 0.95 son columnas que podemos eliminar**

columnas relacion muy baja relacion:


1. onehotencoder__MultipleLines_No phone service =	-0.011942

2. onehotencoder__gender_Male = -0.008612

3. onehotencoder__PhoneService_Yes	= 0.011942

In [239]:
columnas_irrelevantes = ['onehotencoder__MultipleLines_No phone service','onehotencoder__gender_Male','onehotencoder__PhoneService_Yes']
datos_encode_df = datos_encode_df.drop(columns=columnas_irrelevantes)
datos_encode_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 28 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   onehotencoder__Churn_Yes                              7043 non-null   float64
 1   onehotencoder__Partner_Yes                            7043 non-null   float64
 2   onehotencoder__Dependents_Yes                         7043 non-null   float64
 3   onehotencoder__PaperlessBilling_Yes                   7043 non-null   float64
 4   onehotencoder__MultipleLines_Yes                      7043 non-null   float64
 5   onehotencoder__InternetService_Fiber optic            7043 non-null   float64
 6   onehotencoder__InternetService_No                     7043 non-null   float64
 7   onehotencoder__OnlineSecurity_No internet service     7043 non-null   float64
 8   onehotencoder__OnlineSecurity_Yes                     7043

**Calculando VIF, para eliminar las columnas que me generan multicolinealidad

In [240]:
X = datos_encode_df.drop(columns='onehotencoder__Churn_Yes')
y = datos_encode_df['onehotencoder__Churn_Yes']

In [241]:
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]

/usr/local/lib/python3.11/dist-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


In [242]:
print(vif_data.sort_values(by="VIF", ascending=False))

                                             Variable        VIF
10  onehotencoder__DeviceProtection_No internet se...        inf
5                   onehotencoder__InternetService_No        inf
6   onehotencoder__OnlineSecurity_No internet service        inf
8     onehotencoder__OnlineBackup_No internet service        inf
16  onehotencoder__StreamingMovies_No internet ser...        inf
14     onehotencoder__StreamingTV_No internet service        inf
12     onehotencoder__TechSupport_No internet service        inf
25                         remainder__Charges.Monthly  39.678038
26                           remainder__Charges.Total  18.864867
24                                  remainder__tenure  17.854198
4          onehotencoder__InternetService_Fiber optic   9.068195
19                   onehotencoder__Contract_Two year   3.454346
17                 onehotencoder__StreamingMovies_Yes   3.231616
15                     onehotencoder__StreamingTV_Yes   3.184195
2                 onehote

**De acuerdo con lo Anterior, las columnas mayores a 5 representan colinealidad, lo cual nos indica que debemos eliminarlas.**

valors VIF significa que son columnas con multicolinealidad perfectas (elmiminar)

In [244]:
columnas_multicolinealidad = [
    'onehotencoder__DeviceProtection_No internet service',
    'onehotencoder__InternetService_No',
    'onehotencoder__OnlineSecurity_No internet service',
    'onehotencoder__OnlineBackup_No internet service',
    'onehotencoder__StreamingMovies_No internet service',
    'onehotencoder__StreamingTV_No internet service',
    'onehotencoder__TechSupport_No internet service',
    'remainder__Charges.Monthly',
    'remainder__Charges.Total',
    'remainder__tenure',
    'onehotencoder__InternetService_Fiber optic'
]

X = X.drop(columns=columnas_multicolinealidad)
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 16 columns):
 #   Column                                                Non-Null Count  Dtype  
---  ------                                                --------------  -----  
 0   onehotencoder__Partner_Yes                            7043 non-null   float64
 1   onehotencoder__Dependents_Yes                         7043 non-null   float64
 2   onehotencoder__PaperlessBilling_Yes                   7043 non-null   float64
 3   onehotencoder__MultipleLines_Yes                      7043 non-null   float64
 4   onehotencoder__OnlineSecurity_Yes                     7043 non-null   float64
 5   onehotencoder__OnlineBackup_Yes                       7043 non-null   float64
 6   onehotencoder__DeviceProtection_Yes                   7043 non-null   float64
 7   onehotencoder__TechSupport_Yes                        7043 non-null   float64
 8   onehotencoder__StreamingTV_Yes                        7043

### 1.1.3. Load

In [246]:
Datos_transformados = X
Datos_transformados['Churn'] = y
Datos_transformados.to_csv('Datos_transformados.csv', index=False)